In [160]:
import json
import pandas as pd
import importlib
import pickle

In [159]:
filepath = '../scraping/articles/filtered_articles.json'
with open(filepath, 'r') as file:
    articles = pd.read_json(filepath)

filepath = '../relation_extraction/directory.pkl'

with open(filepath, 'rb') as file:
    keywords = pickle.load(file)

In [161]:
import KnowledgeExtractor
importlib.reload(KnowledgeExtractor)

extractor = KnowledgeExtractor.KnowledgeExtractor('dslim/bert-base-NER', keywords)

                ___              __         
               /\_ \      __    /\ \        
 _ __     __   \//\ \    /\_\   \ \ \/'\    
/\`'__\ /'__`\   \ \ \   \/\ \   \ \ , <    
\ \ \/ /\  __/    \_\ \_  \ \ \   \ \ \\`\  
 \ \_\ \ \____\   /\____\  \ \_\   \ \_\ \_\
  \/_/  \/____/   \/____/   \/_/    \/_/\/_/
                                            
                                            



/Users/tiril/Documents/IndividualProject/nuclear_repo/env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[2024-08-06 17:39:23,354] [INFO] [relik.inference.annotator.from_pretrained:694] [PID:29125] [RANK:0] Loading Relik from sapienzanlp/relik-relation-extraction-nyt-large
[2024-08-06 17:39:23,356] [INFO] [relik.inference.annotator.from_pretrained:695] [PID:29125] [RANK:0] {
    '_target_': 'relik.inference.annotator.Relik',
    'index': {
        'triplet': {
            '_target_': 'relik.retriever.indexers.inmemory.InMemoryDocumentIndex.from_pretrained',
            'name_or_path': 'sapienzanlp/relik-retriever-small-nyt-document-index',
        },
    },
    'metadata_fields': [],
    'reader': {
        '_target_': 'relik.reader.pytorch_modules.triplet.RelikReaderForTripletExtraction',
        'transformer_model': 'sapienzanlp/relik-reader-deberta-v3-large-nyt',
    },
    'retriever': {
        'triplet': {
            '_target_': 'relik.retriever.pytorch_modules.model.GoldenRetriever',
            'question_encoder': 'sapienzanlp/relik-retriever-small-nyt-question-encoder',
        

In [165]:
sents = []
if sents:
    print('hello')

In [164]:
output = []
length = len(articles)
for i in range(length):
    if i%20 == 0:
        print(f"{(i/length)*100:.2f}% finished")
    instance = articles.iloc[i]
    url = instance['url']
    sents = instance['text']
    entities = extractor.extract_entities(sents)
    triplets = extractor.extract_relik_triplets(sents, url)
    if triplets:
        output.append({
            'url': url,
            'triplets': triplets,
            'entities': entities
        })

with open('entities_and_triplets.json', 'w') as f:
    json.dump(output, f, indent=4)

with open('untreated_articles.json', 'w') as f:
    json.dump(extractor.untreated_articles, f, indent=4)

[2024-08-06 17:43:12,816] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 42 number of elements processed
[2024-08-06 17:43:43,746] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 19 number of elements processed
[2024-08-06 17:44:09,102] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 16 number of elements processed
[2024-08-06 17:44:31,304] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 29 number of elements processed
[2024-08-06 17:45:00,297] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 19 number of elements processed
[2024-08-06 17:45:26,729] [DEBUG] [relik.reader.data.relik_reader_re_data.__iter__:399] [PID:29125] [RANK:0] Dataset finished: 17 number of elements processed
